<a href="https://colab.research.google.com/github/Andyfer004/Lab-8IA/blob/main/LAB8IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB 8 - Inteligencia Artificial  
## Catedrático: Alberto Suriano  
### Estudiantes:  
- Andy Fuentes  
- Davis Roldán  
- Diederich Solís  



## Tasks 1 - Teoría

### 1. Investigar el algoritmo AC-3 y su relación con el algoritmo de backtracking search

AC-3 (Arc Consistency Algorithm 3) es un algoritmo utilizado en problemas de satisfacción de restricciones (CSP) para reducir los dominios de las variables antes o durante la búsqueda. Lo que hace es asegurarse de que todas las variables estén *arc-consistent*, es decir, que para cada valor de una variable exista al menos un valor compatible en las variables relacionadas.

Su relación con el algoritmo de backtracking search es que AC-3 puede utilizarse como una forma de **pre-procesamiento** o como **heurística** durante la búsqueda. Esto permite reducir significativamente el espacio de búsqueda, ya que elimina valores inconsistentes antes de intentar asignaciones, disminuyendo así el número de retrocesos (backtracks) necesarios.



### 2. Defina en sus propias palabras el término “Arc Consistency”

Una variable es *arc-consistent* con respecto a otra si, para cada valor posible en su dominio, hay al menos un valor en el dominio de la otra variable que satisface la restricción entre ambas. Es decir, no hay valores sin apoyo en las variables relacionadas. El objetivo es que no existan valores que, al asignarse, hagan inevitable una falla más adelante.